In [689]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import fibsem
from fibsem.imaging import masks
from fibsem.imaging import utils
from autoscript_sdb_microscope_client.structures import AdornedImage
from PIL import Image
import logging

import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

from skimage import data
from skimage import filters
from skimage.color import rgb2gray

import imageio
import cv2
import glob
import itertools
from itertools import tee
from itertools import islice

In [690]:
# img1 = mpimg.imread("../pics/1-fl-ZOOM_2.jpg")
# img2 = mpimg.imread("../pics/1-fl-ZOOM_2.jpg")
# img1 = rgb2gray(img1)
# img2 = rgb2gray(img2)

# # img1 = np.array(Image.fromarray(img1).resize(img2.shape))
# # img1 = np.array(img1, dtype=np.float32)
# # img2 = np.array(img2, dtype=np.float32)

# # print(img1.dtype, img2.dtype)

# # img1 = AdornedImage(img1, None)
# # img2 = AdornedImage(img2, None)

# pixelsize_img1 = img1.data.shape
# pixelsize_img2 = img2.data.shape

# img1_mask = masks.create_bandpass_mask(pixelsize_img1, 100, 4, 3)
# img2_mask = masks.create_bandpass_mask(pixelsize_img2, 100, 4, 3)

In [785]:
def xcorr(img1: np.ndarray, img2: np.ndarray, use_mask: bool = True, use_threshold: bool = True ) -> np.ndarray:

    if img1.data.shape != img2.data.shape:
        err = f"Image 1 {img1.data.shape} and Image 2 {img2.data.shape} need to have the same shape"
        logging.error(err)
        raise ValueError(err)


    if use_mask:
        # Create Fourier Transform WITH Bandpass Filter
        pixelsize_img1 = img1.data.shape
        pixelsize_img2 = img2.data.shape
        img1_mask = masks.create_bandpass_mask(pixelsize_img1, 100, 4, 3)
        img2_mask = masks.create_bandpass_mask(pixelsize_img2, 100, 4, 3)
        img1fft = np.fft.ifftshift(img1_mask * np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(img2_mask * np.fft.fftshift(np.fft.fft2(img2.data)))
        assert img1_mask is not None, "Mask1 can not be None when using a Mask"
        assert img2_mask is not None, "Mask2 can not be None when using a Mask"
    
       
       
    else:
        # Create Fourier Transform
        img1fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img2.data)))

    # Do some shady normalization(?)
    n_pixels1 = img1.data.shape[0] * img1.data.shape[1]
    img1fft[0, 0] = 0
    tmp = img1fft * np. conj(img1fft)
    img1fft = n_pixels1 * img1fft / np.sqrt(tmp.sum())
    

    n_pixels2 = img2.data.shape[0] * img2.data.shape[1]
    img2fft[0, 0] = 0
    tmp = img2fft * np. conj(img2fft)
    img2fft = n_pixels2 * img2fft / np.sqrt(tmp.sum())


    if use_threshold:
        #Create threshold (Otsu's method) in Real Space
        real1 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img1fft)))
        real2 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img2fft)))
        otsu1 = filters.threshold_otsu(np.abs(real1))
        otsu2 = filters.threshold_otsu(np.abs(real2))

         #Create segmented binary Img
        binary1 = (real1 > otsu1)*1
        # plt.title("Threshold: >"+str(otsu1))
        # plt.imshow(binary1, cmap='gray')
        # plt.show()
        binary2 = (real2 > otsu2)*1
        # plt.title("Threshold: >"+str(otsu2))
        # plt.imshow(binary2, cmap='gray')
        # plt.show()

        #Back to Fourier Space for cross-correlation
        fft1 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary1)))
        fft2 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary2)))

        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(fft1 * np.conj(fft2))))
        # plt.imshow(corr)
        # plt.show()

        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        #print('Center:', cen, 'Error:', err)
        return valMax
        

    else:
        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(img1fft * np.conj(img2fft))))
        # plt.imshow(corr)
        # plt.show()

        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        return valMax
        #print('Center:', cen, 'Error:', err)
    

In [786]:
# files = glob.glob("../pics/*fl*.jpg")
# for file in files:
#     images = mpimg.imread(file)
#     # Check if Array dimension is 2 (grayscale)
#     if images.ndim == 2:
#         # Add "empty third" dimension (1 by default)
#         images = np.expand_dims(images, axis=2)
        
#     # print(images.shape)

In [787]:
data = []
files = glob.glob("../xcorrtest/*tif")
for file in files:
    img = mpimg.imread(file)
    image = AdornedImage(img, None)
    data.append(image)
arr = np.array(data)


In [793]:
arr1 = arr[0: arr.size: 2]
arr2 = arr[1: arr.size: 2]

for i in range(4):
    
    valMax = xcorr(arr1[i], arr2[i], True, True)
    print(valMax)
    # valMax2 = xcorr(arr1[i], arr2[i], False, True)
    # valMax3 = xcorr(arr1[i], arr2[i], True, False)
    # valMax4 = xcorr(arr1[i], arr2[i], False, False)


19839.0
9249.0
23265.0
16726.0


16726.0

In [ ]:
img4 = AdornedImage(np.roll(img4, shift=(400, 400), axis=(0, 1)))